# GHSC HazDev Summer Python Tutorial Series
## Intro to Testing, Pytest
August 25, 2021


### Why Test?

- Verify behavior
- Prevent regression
- Document usage/requirements
- Refactor with confidence


### Arrange, Act, Assert pattern

#### Arrange

Set up test preconditions

#### Act

Action being tested

#### Assert

Check that action has expected results


In [17]:
def test_pow():
    # arrange
    base, exponent = 2, 3
    expected = 8
    # act
    actual = pow(base, exponent)
    # assert
    assert actual == expected  # comment describing assertion


test_pow()


### Numpy

`numpy.testing` has useful assertion functions for arrays and floating point.

https://numpy.org/doc/stable/reference/routines.testing.html


In [31]:
import numpy
from numpy.testing import assert_allclose


def test_pi():
    expected = 3.14159
    actual = numpy.pi

    assert_allclose(actual, expected, atol=1e-5, rtol=0)


test_pi()

### Types of Tests

- Test complexity increases moving from Unit to Integration, or Integration to End to End.
- Generally more Unit tests than Integration tests, and more Integration tests than End to End tests.


#### Unit

One method or class

#### Integration

Multiple units working together

#### End to End

Full system, possibly with real data


### Mocking

Mocking can help isolate a test environment.

- `unittest.mock`  
  [https://docs.python.org/3/library/unittest.mock.html](https://docs.python.org/3/library/unittest.mock.html)


#### Patching existing functions

- use `patch` decorator
- need to reference where function is imported
- object is passed as positional argument

In [28]:
from unittest import mock
import requests


@mock.patch("requests.get")
def test_requests(mock_get):
    # arrange
    url = "https://earthquake.usgs.gov"
    expected = "hello"
    mock_get.return_value = "hello"
    # act
    actual = requests.get(url)
    # assert
    mock_get.assert_called_with(url)
    assert actual == expected


test_requests()


#### Larger patching example


```
# examples/event_service.py

class EventService:
    url: str

    def __init__(self, url=DEFAULT_URL):
        self.url = url

    def get_events(self) -> List[Dict]:
        response = requests.get(self.url)
        response.raise_for_status()
        geojson = response.json()
        return geojson["features"]
```

In [30]:
from unittest import mock

from examples.event_service import EventService


# patch the requests module used everywhere
@mock.patch("requests.get")
def test_get_events(mock_get):
    """Test that features are returned from json response.

    This test doesn't need to verify requests.get works,
    only that a successful json response is handled correctly.
    """
    # arrange
    mock_response = mock.Mock()
    mock_response.raise_for_status = mock.Mock()
    mock_response.json.return_value = {"features": [{}, {}]}
    mock_get.return_value = mock_response
    service = EventService()
    # act
    events = service.get_events()
    # assert
    mock_get.assert_called_with(service.url)
    assert len(events) == 2


# patch only the requests module imported into examples.event_service
@mock.patch("examples.event_service.requests.get")
def test_get_events_error(mock_get):
    """Test that exception is thrown if there are HTTP errors.

    This test doesn't need to generate an actual error,
    only that the method raises an error.

    NOTE: There are multiple ways an error could be raised,
    relying on "raise_for_status" is implementation specific.
    """
    # arrange
    mock_response = mock.Mock()
    mock_response.raise_for_status.side_effect = ValueError("request error")
    mock_get.return_value = mock_response
    service = EventService()
    # act
    try:
        service.get_events()
        # assert
        assert False  # service should raise exception
    except:
        pass


test_get_events()
test_get_events_error()


### PyTest

- used by numpy/scipy
- conventions for test discovery
  - `test` directory containing all tests
  - folders inside for each folder in package
  - test files start with `test_{module}`
  - test methods start with `test_{description}`
- `assert` statements for assertions
- `fixture`s to define arrange logic outside test

#### Fixture example

In [35]:
from unittest import mock
import pytest
from examples.event_service import EventService


@pytest.fixture
def event_service():
    # set up before test
    service = EventService()
    # yield value
    yield service
    # cleanup after test

@pytest.fixture
def success_response():
    features = [{}, {}]
    mock_response = mock.Mock()
    mock_response.raise_for_status = mock.Mock()
    mock_response.json.return_value = {"features": features}
    return mock_response


@mock.patch("examples.event_service.requests.get")
def test_get_events(mock_get, event_service, success_response):
    """Test that features are returned from json response.

    This test doesn't need to verify requests.get works,
    only that a successful json response is handled correctly.
    """
    # arrange
    mock_get.return_value = success_response
    service = EventService()
    # act
    events = service.get_events()
    # assert
    mock_get.assert_called_with(service.url)
    assert len(events) == 2


### Coverage

- Integrates with pytest using `pytest-cov`
  - `pytest --cov=examples`
- Instruments code to see which lines are executed.
- Coverage is a useful tool, but 100% coverage is possible without useful tests.

### Gitlab Pipelines

- run tests for merge requests
- prevent automated deployment if tests fail


```

Test:
  artifacts:
    reports:
      cobertura: coverage.xml
      junit: junit.xml
  before_script:
    - which python
    # install dependencies or use artifact for virtual environment
    # - poetry install
  image: ${DEVOPS_REGISTRY}usgs/python:3.8-build
  script:
    # run tests
    - pytest --cov=examples --junitxml junit.xml
    # convert pytest-cov output to coverage.xml
    - coverage xml
  stage: test
```


### Strategies


#### Test Driven Development  

Write tests before code, only write code when a test is failing


#### Bug Driven Testing  

Write test to reproduce bug, write code to fix failing test  
  
